In [52]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false
}

<IPython.core.display.Javascript object>

In [54]:
from CoreConfig.CoreConfig import CoreConfig
from DBUtility.DBUtility import DBSQLiteConnection, CreateEventsTableIfNotExist, CreateVenturesTeamTableIfNotExist
from DBUtility.DBUtility import FetchEventsRecordWithinSamePerson, FetchEventsLatestRecordWithSamePerson, FetchEventsRecordAll, FetchVenturesPerson, InsertVenturesPerson
from DBUtility.DBUtility import InsertEventRecord
from pandas import DataFrame
import datetime
CoreConfig.initialize_core_config()

DBSQLiteConnection().configure_connection_information(**CoreConfig().get_sqlite3_db_connection_info())
CreateEventsTableIfNotExist().execute()

CreateVenturesTeamTableIfNotExist().execute()

In [55]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, interactive_output
from IPython.display import display_pretty
import pandas as pd
from ipywidgets import Layout, Button, Box
items_layout = Layout(display='flex',
                    flex_flow='auto',
                     align_items='stretch',
                      width='auto')     # override the default width of the button to 'auto' to let the button grow



box_layout = Layout(display='flex',
                    flex_flow='auto',
                    align_items='center',
                    width='auto')

textarea_layout = Layout(display='flex',
                    flex_flow='auto',
                    align_items='center',
                    width='auto',
                    height='120px')

style = {'description_width': 'initial'}

In [56]:
title = widgets.HTML("<center><H3>Recent records</H3><center>")
formtitle = widgets.HTML("<center><H3>Submit Form</H3><center>")
df_out = widgets.Output(layout={'border': '1px solid black', 'aligh_items' : 'center'})
with df_out:
    df = DataFrame(FetchEventsLatestRecordWithSamePerson().execute(), columns=['Events Name', 'Person', 'Date'])
    display(df)
    
events = widgets.Text(
      value='',
      placeholder='Type Events',
      description='Events:',
      layout=box_layout,
       style=style,
      disable=False)

meeting = widgets.Checkbox(
    value=False,
    description='Board Meeting',
    disabled=False
)

date = widgets.DatePicker(
        value = datetime.datetime.now().date(),
        description='Pick a Date',
        layout=box_layout,
        style = style,
        disable=False)

name=widgets.Text(
      value='',
      placeholder='Type speaker\'s name',
      description='Speaker:',
      layout=box_layout,
       style=style,
      disable=False)
#display_pretty(name)
notes=widgets.Text(
      value='',
      placeholder='Type note/organizer',
      description='Note/ Organizer:',
      layout=box_layout,
       style=style,
      disable=False)

country = widgets.Dropdown(options= {'Australia': 'Australia',
                                     'New Zealand': 'New Zealand',
                                    'None':'None'},
                           value = 'Australia',
                           description='Select country:',
                           layout=box_layout,
                           style=style)
audience= widgets.Text(value='0',
                       placeholder='Type audience numbers',
                       description='Audience:',
                       layout=box_layout,
                       style=style,
                       disable=False)


website= widgets.Text(value='https://',
                       placeholder='Type website/context',
                       description='Website:',
                       layout=box_layout,
                       style=style,
                       disable=False)
asset = widgets.Text(value='',
                       placeholder='Type asset',
                       description='Asset:',
                       layout=box_layout,
                       style=style,
                       disable=False)


person_list = FetchVenturesPerson().execute()
person_dict = {v[0]:v[0] for i, v in enumerate(person_list)}
person = widgets.Dropdown(
                options= person_dict,
                description='Select your person:',
                layout=box_layout,
                style=style
              )

new_person_check = widgets.Checkbox(
    value=False,
    description='Add new person',
    disable = False
)
new_person_text = widgets.Text(
      value='',
      placeholder='Type your name',
      description='New person to be added:',
      layout=box_layout,
       style=style,
      disabled=True)

new_person_button = widgets.Button(
    description='Submit',
    disabled=True,
    button_style ='', # success
    tooltip='Click me',
    icon='check')



new_person_box = widgets.HBox([new_person_check, new_person_text, new_person_button])

button = widgets.Button(
           description='Submit the form',
           disable=False,
           button_style ='', # success
           tooltip='Click me',
           icon='check'
           )

def on_add_new_name_value_change(change):
    if change['new'] == True:
        new_person_text.disabled=False
        new_person_button.disabled=False
    else:
        new_person_text.disabled=True
        new_person_button.disabled=True


def on_submit_add_new_name_button_clicked(b):
    '''Check the local person_dict, if not included, 
    then added and refresh
    '''
    temp_variable = person.options.copy()
    new_person = new_person_text.value
    temp_variable[new_person] = new_person
    person.options = temp_variable
    new_person_check.value = False
    new_person_text.disabled=True
    new_person_button.disabled=True
    InsertVenturesPerson(new_person).execute()

new_person_check.observe(on_add_new_name_value_change, names='value')
new_person_button.on_click(on_submit_add_new_name_button_clicked)
def on_submit_form_button_clicked(b):
    outputs = {'events': [events.value],
                'meeting' : [meeting.value],
                'date': [date.value],
                'name': [name.value],
                'notes': [notes.value],
                'country': [country.value],
                'audience': [audience.value], 
                'website': [website.value],
                'asset' : [asset.value],
                'person' : [person.value]
               }
    InsertEventRecord(events.value, meeting.value, date.value, name.value, notes.value,
                      country.value, audience.value, website.value, asset.value, person.value).execute()
    df = DataFrame(FetchEventsLatestRecordWithSamePerson().execute(), columns=['Events Name', 'Person', 'Date'])
    df_out.clear_output()
    with df_out:
        display(df)
button.on_click(on_submit_form_button_clicked)

In [57]:
def form(events, meeting, date, name, notes, country, audience, website, asset, person, new_person_name):
    #display(events, meeting, date, name, notes, country, audience, website, asset, person)
    return events, meeting, date, name, notes, country, audience, website, asset, person, new_person_name

In [58]:
ui = widgets.VBox(children=[title,
                            df_out,
                            formtitle,
                            events,
                            meeting,
                            date,
                            name,
                            notes,
                            country,
                            audience,
                            website,
                            asset,
                            person,
                            button,
                            new_person_box], layout=items_layout)

In [59]:
out = interactive_output(form, {'events': events,
                                'meeting' : meeting,
                                'date':date,
                                'name':name,
                                'notes':notes,
                                'country':country,
                                'audience': audience, 
                                'website': website,
                                'asset' : asset,
                                'person' : person,
                                'new_person_name' : new_person_box.children[1]
                               })
display(ui, out)


Output()